In [1]:
import os, sys
import torch
import torch.nn as nn
from torchvision import models
from py_auto_fact import auto_fact

In [2]:
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())

# Init Model

In [3]:
model = models.vgg16(pretrained=True)
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# Factorize Model

In [4]:
count_param(model)

138357544

### Apply absolute rank

In [5]:
%%time
fact_model = auto_fact(model, rank=256, deepcopy=True, solver='random', eigen_threshold=0.6)
count_param(fact_model)

TypeError: auto_fact() got an unexpected keyword argument 'eigen_threshold'

In [6]:
%%time
fact_model = auto_fact(model, rank=256, deepcopy=True, solver='svd', eigen_threshold=0.6)
count_param(fact_model)

TypeError: auto_fact() got an unexpected keyword argument 'eigen_threshold'

In [7]:
%%time
fact_model = auto_fact(model, rank=256, deepcopy=True, solver='snmf', eigen_threshold=0.6)
count_param(fact_model)

TypeError: auto_fact() got an unexpected keyword argument 'eigen_threshold'

In [8]:
%%time
fact_model = auto_fact(model, rank=256, deepcopy=True, solver='random', num_iter=50, eigen_threshold=0.0)
count_param(fact_model)

TypeError: auto_fact() got an unexpected keyword argument 'eigen_threshold'

In [9]:
%%time
fact_model = auto_fact(model, rank=256, deepcopy=True, solver='svd', num_iter=50, eigen_threshold=0.0)
count_param(fact_model)

TypeError: auto_fact() got an unexpected keyword argument 'eigen_threshold'

In [10]:
%%time
fact_model = auto_fact(model, rank=256, deepcopy=True, solver='snmf', num_iter=50, eigen_threshold=0.0)
count_param(fact_model)

TypeError: auto_fact() got an unexpected keyword argument 'eigen_threshold'

### Apply percentage rank

In [11]:
%%time
fact_model = auto_fact(model, rank=0.4, deepcopy=True, solver='random', num_iter=50)
count_param(fact_model)

CPU times: user 1.1 s, sys: 136 ms, total: 1.24 s
Wall time: 417 ms


52613904

In [12]:
%%time
fact_model = auto_fact(model, rank=0.4, deepcopy=True, solver='svd', num_iter=50)
count_param(fact_model)

CPU times: user 12min 58s, sys: 1min 22s, total: 14min 21s
Wall time: 1min 41s


52613904

In [13]:
%%time
fact_model = auto_fact(model, rank=0.4, deepcopy=True, solver='snmf', num_iter=50)
count_param(fact_model)

CPU times: user 12min 23s, sys: 1min 19s, total: 13min 43s
Wall time: 1min 29s


52613904

# Test on CPU

### Test Inference CPU

In [14]:
%%timeit
with torch.no_grad():
    y = model(torch.zeros(16,3,224,224, dtype=torch.float))

/home/samuel/anaconda3/envs/deep_fact/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


403 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit
with torch.no_grad():
    y = fact_model(torch.zeros(16,3,224,224, dtype=torch.float))

256 ms ± 3.25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Test Forward-Backward CPU

In [16]:
%%timeit
y = model(torch.zeros(8,3,224,224, dtype=torch.float))
y.sum().backward()

675 ms ± 800 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
y = fact_model(torch.zeros(8,3,224,224, dtype=torch.float))
y.sum().backward()

376 ms ± 779 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Test on GPU

### Move models to GPU

In [18]:
model = model.cuda()
fact_model = fact_model.cuda()

### Test Inference GPU

In [19]:
x = torch.zeros(16,3,224,224, dtype=torch.float).cuda()

In [20]:
%%timeit
with torch.no_grad():
    y = model(x)

41.2 ms ± 183 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
%%timeit
with torch.no_grad():
    y = fact_model(x)

25.7 ms ± 17.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Test Forward-Backward GPU

In [22]:
x = torch.zeros(8,3,224,224, dtype=torch.float).cuda()

In [ ]:
%%timeit
y = model(x)
y.sum().backward()

In [ ]:
%%timeit
y = fact_model(x)
y.sum().backward()